In [1]:
from pathlib import Path
import re
import pandas as pd
from IPython.display import Audio, display
from pydub import AudioSegment
import numpy as np
from textwrap import wrap
from time import sleep

In [9]:
pattern = re.compile(r"^[sS]tortinget-(\d{4}).*")

def prepare_segment(row, audiofolder):
    audio = list(row.audiofilename)[0] + ".mp3"
    year = pattern.match(audio).group(1)
    segment = AudioSegment.from_mp3(Path(audiofolder) / year / audio)
    segment.export("test_clip.wav", format='wav')
    print(f"File: {audio}\n")
    print(f"Language: {list(row.language)[0]}")
    print("corpus:\n" + '\n'.join(wrap(list(row.proceedings_text)[0], width=100)) + "\n")
    print(f"asr:\n" + '\n'.join(wrap(list(row.transcription_text)[0], width=100)) + "\n")


In [3]:
audiofolder = "/media/pers/elements/npscpluss_audio/segmented"

In [4]:
annotation_df = pd.read_csv("data/annotated_dataset.csv")

In [5]:
def get_annotation(row):
    row.missing_start_word = False
    row.missing_end_word = False
    row.wrong_match = False
    row.wrong_language = False
    row.low_quality = False
    if input("Missing start word?") != "":
        row.missing_start_word = True
    if input("Missing end word?") != "":
        row.missing_end_word = True
    if input("Wrong match?") != "":
        row.wrong_match = True
    if input("Wrong language?") != "":
        row.wrong_language = True
    if input("Low quality?") != "":
        row.low_quality = True
    return row

In [6]:
def annotate_data(filename, audiofolder):
    df = pd.read_csv(filename)
    updated_cells = [
        "missing_start_word",
        "missing_end_word",
        "wrong_match",
        "wrong_language",
        "low_quality",
    ]
    subset_df = df[df.missing_start_word.isna()]
    random_row = subset_df.sample()
    prepare_segment(random_row, audiofolder)
    display(Audio("test_clip.wav", autoplay=True))
    random_row = get_annotation(random_row)
    df.loc[random_row.index] = random_row
    df.to_csv(filename, index=False)


    

In [8]:
annotate_data("data/annotated_dataset.csv", audiofolder)

File: Stortinget-20200312-095501_5988400_6016000.mp3

score: 0.8201438848920863
Language: bm
corpus:
næringsliv i hele Norge. Med tanke på handel, næringsliv og arbeidsplasser rundt omkring i
distriktene er vi avhengig av tilgangen til det europeiske markedet som vi får gjennom EØS-avtalen.
Det gir oss tilgang til et marked for våre varer, og det gir oss tilgang på forsyninger som vi ser
særlig effektene av, og som er truede og sårbare, i den situasjonen vi er i i dag.

asr:
næringsliv eee i hele norge med tanke på handel næringsliv og arbeidsplasser rundt omkring i
distriktene så er vi avhengig av tilgangen til det europeiske markedet som vi får eee her hjemme
gjennom eøsavtalen det gir oss tilgang til et marked med våre varer og det gir oss også tilgang til
å få en forsyning som vi ser særlig effektene av som kan være truete og sårbare i den sen som vi er
i dag

